In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys

import numpy as np
import pandas as pd
import plotly as pl

In [ ]:
sys.path.insert(0, "..")

import ccal

np.random.random(20121020)

pl.offline.init_notebook_mode(connected=True)

In [ ]:
for arrar_1d in (
    np.random.random_sample(8),
    np.random.random_sample(16),
    np.random.random_sample(32),
    np.random.random_sample(64),
    np.random.random_sample(128),
    np.random.random_sample(256),
    np.random.random_sample(512),
    np.random.random_sample(1024),
    np.random.random_sample(1000000),
    np.asarray((0,) * 100),
    np.asarray((1,) * 100),
    np.asarray((1,) * 1000),
    np.asarray((1,) * 10000),
    np.asarray((1,) * 100000),
    np.asarray((1,) * 1000000),
):

    entropy = ccal.compute_entropy(arrar_1d)

    print(entropy)

In [ ]:
for x, y in (
    (np.asarray((0,) * 8),) * 2,
    (np.asarray((1,) * 8),) * 2,
    (np.asarray(range(8)), np.asarray(range(8))[::-1]),
    (np.asarray((0, 1, 2, 3, 4, 5, 6, 7, 8, 7, 6, 5, 4, 3, 2, 1, 0)),) * 2,
    (np.asarray((8, 7, 6, 5, 4, 3, 2, 1, 0, 1, 2, 3, 4, 5, 6, 7, 8)),) * 2,
    (np.asarray((0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0)),) * 2,
    (
        np.asarray((1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1)),
        np.asarray((0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0)),
    ),
):

    print(0)

    print(x)

    print(y)

    information_coefficient = ccal.compute_information_coefficient(x, y)

    information_distance = ccal.compute_information_distance(x, y)

    print(
        "Information Coefficient = {:.3f}\tInformation Distance = {:.3f}".format(
            information_coefficient, information_distance
        )
    )

In [ ]:
n = 3

x = np.asarray((0,) * n + (1,) * n + (2,) * n)

print(x)

for factor in (0.1, 0.5, 1, 2, 10, 100):

    y = x * factor

    print()

    print(y)

    ic = ccal.compute_information_coefficient(x, y)

    print(ic)